In [9]:
from energy_model.configs.columns import SystemColumns, ProcessColumns
from energy_model.pipelines.pipeline_utils import extract_x_y
import pandas as pd
from typing import Any
import numpy as np

from energy_model.energy_model_parameters import PROCESS_SYSTEM_DF_PATH, SYSTEM_ONLY_DF_PATH
from energy_model.pipelines.grid_search_pipeline_executor import GridSearchPipelineExecutor

from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import BayesianRidge
from sklearn.cross_decomposition import PLSRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ARDRegression
from sklearn.linear_model import HuberRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import RANSACRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import Lars
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.linear_model import LarsCV
from sklearn.linear_model import LassoCV
from sklearn.linear_model import LassoLars
from sklearn.linear_model import LassoLarsCV
from sklearn.linear_model import LassoLarsIC
from sklearn.svm import LinearSVR
from sklearn.linear_model import RidgeCV
from sklearn.compose import TransformedTargetRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.linear_model import SGDRegressor


# Models

In [10]:
AdaBoostRegressorModel = {"classifier": [AdaBoostRegressor()],
                          "classifier__learning_rate": [0.01, 0.1],
                          "classifier__n_estimators": [50, 100, 500]}

GradientBoostingRegressorModel = {"classifier": [GradientBoostingRegressor()],
                                  'classifier__max_depth': [80, 110],
                                  'classifier__max_features': [3],
                                  'classifier__min_samples_leaf': [3, 5],
                                  'classifier__min_samples_split': [8, 12],
                                  'classifier__n_estimators': [100, 500, 1000]}

ExtraTreeRegressorModel = {"classifier": [ExtraTreeRegressor()],
                           # 'classifier__n_estimators': [10, 50, 100],
                           'classifier__criterion': ['squared_error', 'absolute_error'],
                           'classifier__max_depth': [2, 16,50],
                           'classifier__min_samples_split': [2, 6],
                           'classifier__min_samples_leaf': [1, 2],
                           # 'oob_score': [True, False],
                           'classifier__max_features': ['auto', 'sqrt']}
ElasticNetModel = {"classifier": [ElasticNet()],
                   "classifier__max_iter": [5, 50],
                   "classifier__alpha": [0.001, 0.01, 0.1],
                   "classifier__l1_ratio": np.arange(0.0, 1.0, 0.1)}

HistGradientBoostingRegressorModel = {"classifier": [HistGradientBoostingRegressor()],
                                      'classifier__max_depth': [5, 10, 16],#range(5, 16, 2),
                                      'classifier__min_samples_leaf': [10, 50, 100]} #range(10, 100, 10)}
ExtraTreesRegressorModel = {"classifier": [ExtraTreesRegressor()],
                            "classifier__max_depth": [3, 5, 7, 12],
                            "classifier__min_samples_leaf": [3, 7],
                            "classifier__min_weight_fraction_leaf": [0.1, 0.5],
                            "classifier__max_features": ["auto", "sqrt"],
                            "classifier__max_leaf_nodes": [10, 60, 90]}

RandomForestRegressorModel = {"classifier": [RandomForestRegressor()],
                              'classifier__n_estimators': [50, 100, 500, 1000],
                              'classifier__max_features': ['auto', 'sqrt'],
                              'classifier__max_depth': [5, 7, 15, 60],
                              'classifier__min_samples_split': [2, 5, 10],
                              'classifier__min_samples_leaf': [1, 4]}

In [11]:
all_possible_models = {
    "AdaBoostRegressorModel": AdaBoostRegressorModel,
    "GradientBoostingRegressorModel": GradientBoostingRegressorModel,
    "ExtraTreesRegressorModel": ExtraTreesRegressorModel,
    "ExtraTreeRegressorModel": ExtraTreeRegressorModel,
    "ElasticNetModel": ElasticNetModel,
    "HistGradientBoostingRegressorModel": HistGradientBoostingRegressorModel,
    "RandomForestRegressorModel": RandomForestRegressorModel
}

In [12]:
process_df_path = PROCESS_SYSTEM_DF_PATH
system_only_df_path = SYSTEM_ONLY_DF_PATH

system_target = SystemColumns.ENERGY_USAGE_SYSTEM_COL
process_target = ProcessColumns.ENERGY_USAGE_PROCESS_COL

In [13]:
def run_grid_search(target_col: str, dataset_path: str, possible_models: list[dict]) -> dict[str, Any]:
    grid_search_pipeline = GridSearchPipelineExecutor(possible_models=possible_models)
    dataset = pd.read_csv(dataset_path, index_col=0)
    X, y = extract_x_y(dataset, target_column=target_col)
    best_model = grid_search_pipeline.run_grid_search(X, y)
    return best_model

In [14]:
def print_best_models(best_model: dict[str, Any], model_name: str) -> str:
    res = f"\n\nGrid Search Results for Model {model_name}: \n{best_model}"
    print(res)
    return res

In [15]:
def run_grid_search_on_all_models(target_col: str, dataset_path: str, model_options: dict[str, dict[str, Any]]) -> dict[str, dict[str, Any]]:
    best_model_per_type = {}
    full_results = ""
    for model_name, model in model_options.items():
        full_results += f"\n\n***** Starting Grid Search for Model {model_name}: *****\n"
        print(f"\n\n***** Starting Grid Search for Model {model_name}: *****\n")
        best_model = run_grid_search(target_col, dataset_path, [model])
        res = print_best_models(best_model, model_name)
        full_results += res
        best_model_per_type[model_name] = best_model
        print(f"\n\n***** Finished Grid Search for Model {model_name}: *****\n")
        full_results += f"\n\n***** Finished Grid Search for Model {model_name}: *****\n"

    with open("full_results.txt", "w") as f:
        f.write(full_results)
    return best_model_per_type


# Find best system energy model

In [8]:
best_system_models = run_grid_search_on_all_models(system_target, system_only_df_path, all_possible_models)



***** Starting Grid Search for Model AdaBoostRegressorModel: *****

*** Select best 5 regressors + their params: 
Fitting 5 folds for each of 6 candidates, totalling 30 fits
~~~~~~~~~~~~~~~ Best 5 models based on neg_mean_absolute_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_mean_absolute_error on train: 2.95031727085751
model params: {'classifier': AdaBoostRegressor(), 'classifier__learning_rate': 0.01, 'classifier__n_estimators': 50}

*************** model number 2 ***************
model neg_mean_absolute_error on train: 2.9736961942069517
model params: {'classifier': AdaBoostRegressor(), 'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100}

*************** model number 3 ***************
model neg_mean_absolute_error on train: 3.012604406073562
model params: {'classifier': AdaBoostRegressor(), 'classifier__learning_rate': 0.1, 'classifier__n_estimators': 50}

*************** model number 4 ***************
model neg_mean_absolute

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
160 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

~~~~~~~~~~~~~~~ Best 5 models based on neg_mean_absolute_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_mean_absolute_error on train: 12.443671647216533
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 3, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 60, 'classifier__min_samples_leaf': 3, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 2 ***************
model neg_mean_absolute_error on train: 12.513140379491343
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 5, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 90, 'classifier__min_samples_leaf': 3, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 3 ***************
model neg_mean_absolute_error on train: 12.674077492726813
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 5, 'classifier__max_features': 'sqrt', 'classifier__max_le

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
206 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

~~~~~~~~~~~~~~~ Best 5 models based on neg_root_mean_squared_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_root_mean_squared_error on train: 13.52347794830917
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 3, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 10, 'classifier__min_samples_leaf': 7, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 2 ***************
model neg_root_mean_squared_error on train: 13.678237125399864
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 5, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 90, 'classifier__min_samples_leaf': 3, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 3 ***************
model neg_root_mean_squared_error on train: 13.944107592251314
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'cl

...

~~~~~~~~~~~~~~~ Best 5 models based on neg_root_mean_squared_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_root_mean_squared_error on train: 5.256704595626447
model params: {'classifier': ElasticNet(), 'classifier__alpha': 0.01, 'classifier__l1_ratio': np.float64(0.0), 'classifier__max_iter': 50}

*************** model number 2 ***************
model neg_root_mean_squared_error on train: 5.256788811365764
model params: {'classifier': ElasticNet(), 'classifier__alpha': 0.01, 'classifier__l1_ratio': np.float64(0.1), 'classifier__max_iter': 50}

*************** model number 3 ***************
model neg_root_mean_squared_error on train: 5.256880664619002
model params: {'classifier': ElasticNet(), 'classifier__alpha': 0.01, 'classifier__l1_ratio': np.float64(0.2), 'classifier__max_iter': 50}

*************** model number 4 ***************
model neg_root_mean_squared_error on train: 5.25698113558268
model params: {'classifier': ElasticNet(), 'classifier__

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
450 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

~~~~~~~~~~~~~~~ Best 5 models based on neg_mean_absolute_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_mean_absolute_error on train: 1.0315671013725851
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 500}

*************** model number 2 ***************
model neg_mean_absolute_error on train: 1.0348449678555232
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1000}

*************** model number 3 ***************
model neg_mean_absolute_error on train: 1.0416705611722432
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
297 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

~~~~~~~~~~~~~~~ Best 5 models based on neg_root_mean_squared_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_root_mean_squared_error on train: 1.9309089938611241
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 500}

*************** model number 2 ***************
model neg_root_mean_squared_error on train: 1.9349024871960547
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1000}

*************** model number 3 ***************
model neg_root_mean_squared_error on train: 1.9479978878830206
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__

# Find Best Process Energy Model

In [16]:
best_process_models = run_grid_search_on_all_models(process_target, process_df_path, all_possible_models)



***** Starting Grid Search for Model AdaBoostRegressorModel: *****

*** Select best 5 regressors + their params: 
Fitting 5 folds for each of 6 candidates, totalling 30 fits
~~~~~~~~~~~~~~~ Best 5 models based on neg_mean_absolute_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_mean_absolute_error on train: 1.069899666548864
model params: {'classifier': AdaBoostRegressor(), 'classifier__learning_rate': 0.1, 'classifier__n_estimators': 50}

*************** model number 2 ***************
model neg_mean_absolute_error on train: 1.072014303884581
model params: {'classifier': AdaBoostRegressor(), 'classifier__learning_rate': 0.01, 'classifier__n_estimators': 500}

*************** model number 3 ***************
model neg_mean_absolute_error on train: 1.080006014009471
model params: {'classifier': AdaBoostRegressor(), 'classifier__learning_rate': 0.01, 'classifier__n_estimators': 100}

*************** model number 4 ***************
model neg_mean_absolut

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

~~~~~~~~~~~~~~~ Best 5 models based on neg_mean_absolute_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_mean_absolute_error on train: 1.4419534036745147
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 90, 'classifier__min_samples_leaf': 7, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 2 ***************
model neg_mean_absolute_error on train: 1.4424462136572287
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 90, 'classifier__min_samples_leaf': 3, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 3 ***************
model neg_mean_absolute_error on train: 1.44887399985137
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 3, 'classifier__max_features': 'sqrt', 'classifier__max_le

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
240 fits failed out of a total of 480.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
96 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  F

~~~~~~~~~~~~~~~ Best 5 models based on neg_root_mean_squared_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_root_mean_squared_error on train: 1.804633892926296
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 90, 'classifier__min_samples_leaf': 7, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 2 ***************
model neg_root_mean_squared_error on train: 1.8077614830382238
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', 'classifier__max_leaf_nodes': 60, 'classifier__min_samples_leaf': 7, 'classifier__min_weight_fraction_leaf': 0.1}

*************** model number 3 ***************
model neg_root_mean_squared_error on train: 1.8135466173843517
model params: {'classifier': ExtraTreesRegressor(), 'classifier__max_depth': 12, 'classifier__max_features': 'sqrt', '

...

~~~~~~~~~~~~~~~ Best 5 models based on neg_root_mean_squared_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_root_mean_squared_error on train: 1.5617016053462311
model params: {'classifier': ElasticNet(), 'classifier__alpha': 0.01, 'classifier__l1_ratio': np.float64(0.0), 'classifier__max_iter': 50}

*************** model number 2 ***************
model neg_root_mean_squared_error on train: 1.5617315923866795
model params: {'classifier': ElasticNet(), 'classifier__alpha': 0.001, 'classifier__l1_ratio': np.float64(0.1), 'classifier__max_iter': 50}

*************** model number 3 ***************
model neg_root_mean_squared_error on train: 1.5617316017205702
model params: {'classifier': ElasticNet(), 'classifier__alpha': 0.001, 'classifier__l1_ratio': np.float64(0.2), 'classifier__max_iter': 50}

*************** model number 4 ***************
model neg_root_mean_squared_error on train: 1.5617319549732211
model params: {'classifier': ElasticNet(), 'class

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
308 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

~~~~~~~~~~~~~~~ Best 5 models based on neg_mean_absolute_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_mean_absolute_error on train: 0.5179464010315484
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1000}

*************** model number 2 ***************
model neg_mean_absolute_error on train: 0.5187356153798529
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 500}

*************** model number 3 ***************
model neg_mean_absolute_error on train: 0.5216838027351905
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf

C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py:516: FitFailedWarning: 
480 fits failed out of a total of 960.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
428 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\model_selection\_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  

~~~~~~~~~~~~~~~ Best 5 models based on neg_root_mean_squared_error score ~~~~~~~~~~~~~~~
*************** model number 1 ***************
model neg_root_mean_squared_error on train: 0.8632776810462897
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 500}

*************** model number 2 ***************
model neg_root_mean_squared_error on train: 0.8660347008482446
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__min_samples_leaf': 1, 'classifier__min_samples_split': 2, 'classifier__n_estimators': 1000}

*************** model number 3 ***************
model neg_root_mean_squared_error on train: 0.8727359271811241
model params: {'classifier': RandomForestRegressor(), 'classifier__max_depth': 60, 'classifier__max_features': 'sqrt', 'classifier__